In [1]:
# Import Dependencies and setup
import os
import pandas as pd
import time
import requests
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
# import psycopg2   # All SQL Loads should be done in a new .ipynb file

##### Extraction

In [2]:
# Extract CSV into pandas df
df_ca = pd.read_csv('../resources/california-history.csv')
df_ca.head()
df_ca =len(df_ca)
print(df_ca)
# df_ca.dtypes  -- used to check data type. Uncomment to check

369


##### Generate List of Data Fields

In [3]:
# Create json url query and variable for data request
base_url = 'https://api.covidtracking.com/v1/states/ny/daily.json'
data_dict = requests.get(base_url).json()

print(json.dumps(data_dict, indent=4, sort_keys=True))
len(data_dict)

[
    {
        "checkTimeEt": "03/06 09:00",
        "commercialScore": 0,
        "dataQualityGrade": null,
        "date": 20210307,
        "dateChecked": "2021-03-06T14:00:00Z",
        "dateModified": "2021-03-06T14:00:00Z",
        "death": 39029,
        "deathConfirmed": null,
        "deathIncrease": 59,
        "deathProbable": null,
        "fips": "36",
        "grade": "",
        "hash": "8b1ad6376deb03844d4e5e3f615197da1b3a214e",
        "hospitalized": null,
        "hospitalizedCumulative": null,
        "hospitalizedCurrently": 4789,
        "hospitalizedDischarged": 150100,
        "hospitalizedIncrease": 0,
        "inIcuCumulative": null,
        "inIcuCurrently": 999,
        "lastUpdateEt": "3/6/2021 14:00",
        "negative": null,
        "negativeIncrease": 0,
        "negativeRegularScore": 0,
        "negativeScore": 0,
        "negativeTestsAntibody": null,
        "negativeTestsPeopleAntibody": null,
        "negativeTestsViral": null,
        "onVentila

371

In [4]:
# Set up dictionary of variables where data from api calls will be stored
ny_data = {'date':[], 'state':[], 'deaths':[], 
           'Icu hospitalized':[], 'positive cases viral':[], 'positive increase':[], 
           'test results total':[], 'test increase':[]
          }

print('Retrieving NY Covid data')
print('-' * 30)

# forLoop to collect data and append to data_dict
for data in data_dict:
#     try:  -- not working; keep to use in future
        ny_data['date'].append(data['date'])
        ny_data['state'].append(data['state'])
        ny_data['deaths'].append(data['deathIncrease'])
#         ny_data['daily_hospitalization'].append(data['hospitalizedIncrease'])
        ny_data['Icu hospitalized'].append(data['inIcuCurrently'])
        ny_data['positive cases viral'].append(data['positiveCasesViral'])
        ny_data['positive increase'].append(data['positiveIncrease'])
        ny_data['test results total'].append(data['totalTestResults'])
        ny_data['test increase'].append(data['totalTestResultsIncrease'])

        # Use timer to delay request to not exceed query limits.
        time.sleep(.075)
#     except:   -- not working; keep to use in future
#         print(f'Incomplete record for {data}. Skipping {data}.')

#         print(ny_data)  -- used to validate append to ny_data dictionary; uncomment to test

Retrieving NY Covid data
------------------------------


In [5]:
# Convert panda dictionary to dataframe and export to csv
ny_df = pd.DataFrame.from_dict(ny_data)

ny_df['date'] = pd.to_datetime(ny_df['date'], format='%Y%m%d')
ny_df.to_csv('../resources/ny_data.csv')
ny_df.head()
# ny_df.dtypes  -- used to check data type. Uncomment to check

,date,state,deaths,Icu hospitalized,positive cases viral,positive increase,test results total,test increase
0,2021-03-07,NY,59,999.0,None,6789,39695100,227768
1,2021-03-06,NY,79,1012.0,None,7647,39467332,273132
2,2021-03-05,NY,95,1030.0,None,8956,39194200,296935
3,2021-03-04,NY,61,1043.0,None,7593,38897265,270089
4,2021-03-03,NY,75,1047.0,None,7704,38627176,218069


In [6]:
# Read in Cleaned CA Data

clean_ca_df = pd.read_csv('../resources/clean_ca_data.csv', index_col=0)

clean_ca_df.head()
# clean_ca_df.dtypes

,date,state,deaths,Icu hospitalized,positive cases viral,positive increase,test results total,test increase
0,2021-03-07,CA,258,1159.0,3501394,3816,49646014,133186
1,2021-03-06,CA,418,1221.0,3497578,4452,49512828,218325
2,2021-03-05,CA,400,1236.0,3493126,4659,49294503,146818
3,2021-03-04,CA,273,1327.0,3488467,3504,49147685,119637
4,2021-03-03,CA,278,1403.0,3484963,3352,49028048,130858


In [7]:
# Merge CA and NY Data on Date
clean_ca_df['date'] = pd.to_datetime(clean_ca_df['date'], errors="coerce")
# CA_NY_Data = pd.merge(ny_df, clean_ca_df, how="inner", on="date")
CA_NY_Data = pd.concat([ny_df, clean_ca_df]).sort_index(kind="merge")
CA_NY_Data


,date,state,deaths,Icu hospitalized,positive cases viral,positive increase,test results total,test increase
0,2021-03-07,NY,59,999.0,None,6789,39695100,227768
0,2021-03-07,CA,258,1159.0,3501394,3816,49646014,133186
1,2021-03-06,NY,79,1012.0,None,7647,39467332,273132
1,2021-03-06,CA,418,1221.0,3497578,4452,49512828,218325
2,2021-03-05,NY,95,1030.0,None,8956,39194200,296935
...,...,...,...,...,...,...,...,...
367,2020-03-05,CA,0,NaN,53,0,515,0
368,2020-03-04,NY,0,NaN,None,0,10,9
368,2020-03-04,CA,0,NaN,53,0,515,0
369,2020-03-03,NY,0,NaN,None,1,1,1


In [8]:
CA_NY_Data = CA_NY_Data.rename(columns= {"date":"Date",
                                        "state":"State",
                                        "deaths":"Deaths",
                                        "Icu hospitalized":"ICU Hospitalizations",
                                        "positive cases viral": "Positive Viral Cases",
                                        "positive increase": "Positive Increase",
                                        "test results total": "Total Test Results",
                                        "test increase": "Test Increase",
                                        })
# CA_NY_Data = CA_NY_Data.dropna()
# CA_NY_Data = CA_NY_Data.reset_index(drop = True)
CA_NY_Data.fillna(0, inplace=True)

CA_NY_Data

,Date,State,Deaths,ICU Hospitalizations,Positive Viral Cases,Positive Increase,Total Test Results,Test Increase
0,2021-03-07,NY,59,999.0,0,6789,39695100,227768
0,2021-03-07,CA,258,1159.0,3501394,3816,49646014,133186
1,2021-03-06,NY,79,1012.0,0,7647,39467332,273132
1,2021-03-06,CA,418,1221.0,3497578,4452,49512828,218325
2,2021-03-05,NY,95,1030.0,0,8956,39194200,296935
...,...,...,...,...,...,...,...,...
367,2020-03-05,CA,0,0.0,53,0,515,0
368,2020-03-04,NY,0,0.0,0,0,10,9
368,2020-03-04,CA,0,0.0,53,0,515,0
369,2020-03-03,NY,0,0.0,0,1,1,1


##### Scrape iframe

In [9]:
# Retrieve COVID graphs from Syracuse website iframes
# Set url query path
path='https://www.syracuse.com/coronavirus-ny/'
f = urlopen(path)
html = str(f.read())
soup = BeautifulSoup(html, 'html.parser')
txt = soup.find_all('iframe')

for element in txt:
    print(element.attrs["src"])

https://datawrapper.dwcdn.net/9TbbF/2/
https://datawrapper.dwcdn.net/ijEiy/2/
https://datawrapper.dwcdn.net/A8g9w/2/
https://datawrapper.dwcdn.net/Scq2a/1/
https://datawrapper.dwcdn.net/pBpl6/1/
//syracusecom.maps.arcgis.com/apps/Embed/index.html?webmap=4ee7a49ca08e42a780d6f8026fca1e5c&extent=-80.0507,41.3424,-71.9372,44.456&home=true&zoom=true&previewImage=false&scale=false&disable_scroll=true&theme=light
https://datawrapper.dwcdn.net/eiIvo/10/
//syracusecom.maps.arcgis.com/apps/Embed/index.html?webmap=265a07eaa42d41c2a901a2a213aa5f36&extent=-80.049,41.3394,-71.9356,44.4531&home=true&zoom=true&previewImage=false&scale=false&disable_scroll=true&theme=light
https://datawrapper.dwcdn.net/BBXOO/10/
//syracusecom.maps.arcgis.com/apps/Embed/index.html?webmap=594b516c770d4fc39870a38fb2eaea73&extent=-80.0549,41.3393,-71.9415,44.453&home=true&zoom=true&previewImage=false&scale=false&disable_scroll=true&theme=light
https://datawrapper.dwcdn.net/D8hmk/17/


##### Retrieve data from interactive chart

In [10]:
# Copied-pasted https://datawrapper.dwcdn.net/ijEiy/2/, a dynamic webpage 
# into browser and inspected page. Examined Network, then XHR to get the link for CSV data.
url = 'https://static.dwcdn.net/data/ijEiy.csv?v=1623222240000'
proxies = {}
response = requests.get(url=url, proxies=proxies)
with open("../resources/covid_chart_data.csv", "wb") as f:
    f.write(response.content)

In [11]:
CA_NY_Data.set_index('Date', inplace=True)
CA_NY_Data

,State,Deaths,ICU Hospitalizations,Positive Viral Cases,Positive Increase,Total Test Results,Test Increase
Date,,,,,,,
2021-03-07,NY,59,999.0,0,6789,39695100,227768
2021-03-07,CA,258,1159.0,3501394,3816,49646014,133186
2021-03-06,NY,79,1012.0,0,7647,39467332,273132
2021-03-06,CA,418,1221.0,3497578,4452,49512828,218325
2021-03-05,NY,95,1030.0,0,8956,39194200,296935
...,...,...,...,...,...,...,...
2020-03-05,CA,0,0.0,53,0,515,0
2020-03-04,NY,0,0.0,0,0,10,9
2020-03-04,CA,0,0.0,53,0,515,0


In [12]:
CA_NY_Data.to_csv('../resources/CA_NY_Data2.csv')